In [ ]:
%run my_import.py

In [ ]:
def ts_plotly(series_dict, labels=None, title="Time series"):
    fig = go.Figure()
    
    for label, series in series_dict.items():
        fig.add_trace(go.Scatter(x=series.index, y=series.values, mode='lines', name=label))
    
    fig.update_layout(title=title, xaxis_title="Data", yaxis_title="Valore", template="plotly_dark")
    # fig.update_layout(include_plotlyjs='cdn')
    fig.show()

In [ ]:
series_dict = {}
refresh_data = False

#### **WHITE NOISE**

In [ ]:
#1. White Noise
n_years = 10
start_date = "2014-01-01"
series_name = "White Noise"
date_range = pd.date_range(start=start_date, periods=365*n_years, freq='D')
np.random.seed(42)
white_noise = np.random.normal(loc=0, scale=1, size=len(date_range))
series = pd.Series(white_noise, index=date_range)
series_dict[series_name] = series
#series.to_csv('wn.csv', index=False)
#print(series.tail(10))
ts_plotly(series_dict, title=series_name)

Serie storica White Noise dal 2014-01-01 al 2023-12-29

#### **ENGLAND TEMPERATURE** (messo in pausa)

In [ ]:
engTemp = loaded_data['EngTemp']

SONO DIVERSE STAZIONI METEOROLOGICHE DELL'INGHILTERRA CENTRALE, CON LA TEMPERATURA MEDIA (target) CALCOLATA SU DI ESSE. \
CI SONO DEI NAN CHE ANDRANNO GESTITI.\
LA SERIE VA DAL 01/01/1853 FINO AL 20/03/2025

#### **OIL PRICE**

In [ ]:
oil_prices = loaded_data['oil_prices']
oil_prices.reset_index(inplace=True)
oil_prices['date'] = oil_prices['date'].dt.strftime('%Y-%m')
oil_prices.set_index('date', inplace=True)
oil_prices.head()

In [ ]:
print("DIMENSIONE:",oil_prices.shape)
print("NAN:",oil_prices.isna().sum())
print("STATISTICHE DESCRITTIVE:",oil_price.describe())

In [ ]:
#ACF E PACF
plot_acf(target, lags=100)
plot_pacf(target, lags=10)
plt.show()

Serie storica mensile che va da Marzo 1983 ad Aprile 2025. Abbiamo anche le variazioni, ma questa serie è stata pensata come univariata sul prezzo greggio del petrolio.

#### **missing data**: serie univariata delle sales di Favorita

In [ ]:
missing = loaded_data['favorita_train']
missing.set_index("date", inplace=True)
missing = missing[['sales','family','date']]
missing.head()

In [ ]:
#sommo tutte  le vendite per ogni giorno
missing = missing.groupby(['date']).sum().reset_index()
missing.drop(columns=["family"], inplace=True)
missing = missing.rename(columns={"sales": "total_daily_sales"})
missing.set_index("date", inplace=True)
ts_plotly({"sales": missing['total_daily_sales']}, title="Sales")

In [ ]:
#ACF E PACF
plot_acf(missing['total_daily_sales'], lags=50)
plot_pacf(missing['total_daily_sales'], lags=50)
plt.show()

#### **outliers data**: serie univariata delle sales di Favorita

In [ ]:
outl = missing.copy()
print("Dimensione:",outl.shape)
print("Quanti NAN ci sono?:",outl.isna().sum())

In [ ]:
#Creo qualche outlier in maniera randomica
#metto 5 osservazioni a 3 milioni come valore, DA CAMBIARE !!!!
np.random.seed(25)

outl_indices = np.random.choice(outl.index, size=5, replace=False)
outl.loc[outl_indices, 'total_daily_sales'] = 3000000
outl.head()
ts_plotly({"sales": outl['total_daily_sales']}, title="Sales with outliers")
print("Dimensione:",outl.shape)

In [ ]:
#ACF E PACF
plot_acf(outl['total_daily_sales'], lags=40)
plot_pacf(outl['total_daily_sales'], lags=40)
plt.show()

####  **SOLAR ENERGY**

In [ ]:
SolarData = loaded_data['solar_power']
SolarData.set_index('DATE_TIME',inplace=True)

In [ ]:
print("Quanti NAN ci sono?:",SolarData.isna().sum())
SolarData.shape
print(SolarData.corr())

La variabile target sarà la quantità (kW) di corrente **diretta** generata dall'invertitore (pannello --> impianto elettrico), cioè quella in blu. \
Dati ogni 15 minuti, dal 15 Maggio 2020 al 17 Giugno 2020.

In [ ]:
ts_plotly({
    'Ambient Temp': SolarData["AMBIENT_TEMPERATURE"], 
    'Panel Temp': SolarData["MODULE_TEMPERATURE"],
}, title='Temperature ambiente VS Temperatura pannello solare')

Questo dataset verrà utilizzato inizialmente per un'analisi univariata della 'Direct current'. Poi, inseriremo dei regressori, fra cui queste due variabili, e la quantità di irradiazione solare (W/m²)

In [ ]:
#aggiungo ora del giorno
df = SolarData
df_reset = df.reset_index()
df_reset.rename(columns={'index': 'DATE_TIME'}, inplace=True)
df_reset['DATE_TIME'] = pd.to_datetime(df_reset['DATE_TIME'])

# Nuove variabili
df_reset['ora_del_giorno'] = df_reset['DATE_TIME'].dt.hour + 1

df = df_reset.set_index('DATE_TIME')
df.head()
#plot dela potenza DC vs ora del giorno
plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x='ora_del_giorno', y='DC_POWER')
plt.title('DC Power vs Hour of the Day')
plt.xlabel('Hour of the Day')
plt.ylabel('DC Power')
plt.xticks(rotation=45)
plt.grid()
plt.show()

#### **AIR QUALITY** (in pausa)

Con regressori, 7/8k osservazioni

In [ ]:
file_path = 'data/AirQualityUCI.csv'
aqi = pd.read_csv(file_path, sep=';')
aqi.drop(columns=["Unnamed: 15","Unnamed: 16"], inplace = True)

In [ ]:
# concateno Date e Time in un'unica nuova colonna DateTime
aqi['DateTime'] = pd.to_datetime(aqi['Date'] + ' ' + aqi['Time'], format='%d/%m/%Y %H.%M.%S')
# DateTime come indice
aqi.set_index('DateTime', inplace=True)

# converto colonne Date e Time in datetime
aqi['Date'] = pd.to_datetime(aqi['Date'], format='%d/%m/%Y')
aqi['Time'] = pd.to_datetime(aqi['Time'], format='%H.%M.%S').dt.time

# creo variabili ora del giorno e giorno della settimana
aqi['ora_del_giorno'] = pd.to_datetime(aqi['Time'], format='%H:%M:%S').dt.hour + 1
aqi['giorno_settimana'] = aqi['Date'].dt.dayofweek + 1

# droppo colonne Date e Time
aqi.drop(columns=["Date", "Time"], inplace=True)
aqi.index = pd.DatetimeIndex(pd.to_datetime(aqi.index))
#aqi.info()

In [ ]:
#sostituisco le virgole con il punto e converto in float
aqi['CO(GT)'] = aqi['CO(GT)'].astype(str).str.replace(',', '.').astype(float)
aqi['C6H6(GT)'] = aqi['C6H6(GT)'].astype(str).str.replace(',', '.').astype(float)
aqi['T'] = aqi['T'].astype(str).str.replace(',', '.').astype(float)
aqi['RH'] = aqi['RH'].astype(str).str.replace(',', '.').astype(float)
aqi['AH'] = aqi['AH'].astype(str).str.replace(',', '.').astype(float)
#aqi.info()

In [ ]:
ts_plotly({'CO':aqi['CO(GT)']}, title="Air Quality Index")
#i valori -200 sono segnati tali perchè sono NaN

In [ ]:
#elimimo (mando a NaN) tutte le celle che contengono un valore minore di 100
#nessuna variabile può assumere valore minore di 100
aqi1 = aqi[aqi>-100]
#aqi1.head()

In [ ]:
#ts_plotly unico di NOX, NO2 e CO
ts_plotly({
    'NOX(GT)':aqi1['NOx(GT)'],
    'NO2':aqi1['NO2(GT)'],
    'CO':aqi1['CO(GT)']
           }, 
          title="Air Quality")

Serie storica con frequenza oraria dal 10/03/2004 al 04/04/2005. Dobbiamo gestire NaN, e scegliere variabili, indipendenti e dipendente. (PCA?)

#### **F1 Race data**

In [ ]:
#NON LANCIARE
#Questa dovrebbe essere più veloce, comunque 6/7 minuti
fastf1.Cache.enable_cache('cacheF1data')
# Carica la sessione
session = fastf1.get_session(2023, 'Silverstone', 'R')
session.load()

# Crea una lista per raccogliere i dati di tutti i piloti
all_telemetry = []

# Colonne da escludere
columns_to_exclude = ["SessionTime", "DriverAhead", "Time", "Source", "Distance", 
                     "RelativeDistance", "X", "Y", "Z"]

# Itera su tutti i piloti della sessione
for drv in session.drivers:
    driver_name = session.get_driver(drv)["Abbreviation"]
    
    # Ottieni i dati di telemetria
    laps = session.laps.pick_driver(driver_name)
    telemetry = pd.DataFrame()
    
    for _, lap in laps.iterrows():
        lap_telemetry = lap.get_telemetry()
        lap_telemetry["Driver"] = driver_name
        lap_telemetry["LapNumber"] = lap.LapNumber
        
        # Rimuovi le colonne non desiderate se presenti
        for col in columns_to_exclude:
            if col in lap_telemetry.columns:
                lap_telemetry = lap_telemetry.drop(columns=[col])
        
        telemetry = pd.concat([telemetry, lap_telemetry], ignore_index=True)
    
    all_telemetry.append(telemetry)

# Unisci tutti i dati in un unico DataFrame
full_telemetry = pd.concat(all_telemetry, ignore_index=True)

full_telemetry.to_csv("F1_data.csv", index=False)
print(full_telemetry.head())

In [ ]:
f1 = pd.read_csv("data\F1_data.csv")
f1.set_index("Date",inplace=True)
#f1.head()

In [ ]:
#Distanza dal pilota davanti
series_dict = {
    'LEC': f1[f1["Driver"]=="LEC"]["DistanceToDriverAhead"],
    'NOR': f1[f1["Driver"]=="NOR"]["DistanceToDriverAhead"]}
ts_plotly(series_dict, title="Distance to Driver Ahead")

In [ ]:
#RPM del motore
series_dict = {
    'LEC': f1[f1["Driver"]=="LEC"]["RPM"],
    'NOR': f1[f1["Driver"]=="NOR"]["RPM"]}
ts_plotly(series_dict, title="RPM")

Dobbiamo scegliere quale variabile impostare come target. Dati rilevati da sensori all'interno delle autovetture di F1, frequenza variabile dai 50 ai 100 millisecondi.

#### **Electricity demand** ####

In [ ]:
file_path = 'data/ElecDemand.csv'
elec = pd.read_csv(file_path)
elec.dropna(inplace=True)

In [ ]:
def add_midnight_time(date_str):
    if len(date_str) < 19:  # Se la stringa è più corta di "2021-01-01 00:00:00"
        return date_str + " 00:00:00"
    return date_str

elec['Date'] = elec['Date'].apply(add_midnight_time)
elec['Date'] = pd.to_datetime(elec['Date'])
elec.set_index('Date',inplace=True)

In [ ]:
series_dict = {
    'UK Elec Demand': elec["Demand"],
    'Workday': elec["Workday"],
    'Temp': elec["Temperature"]
}
ts_plotly(series_dict, title="Elec Demand")  

In [ ]:
#adding features
df_reset = elec.reset_index()
df_reset.rename(columns={'Date':'DATE_TIME'}, inplace=True)
df_reset['DATE_TIME'] = pd.to_datetime(df_reset['DATE_TIME'])

df_reset['giorno_settimana'] = df_reset['DATE_TIME'].dt.dayofweek + 1
df_reset['ora_del_giorno'] = df_reset['DATE_TIME'].dt.hour + 1
df_reset['settimana_del_mese'] = ((df_reset['DATE_TIME'].dt.day - 1) // 7) + 1
df_reset['settimana_del_anno'] = df_reset['DATE_TIME'].dt.isocalendar().week
df_reset['giorno_del_mese'] = df_reset['DATE_TIME'].dt.day
df_reset['mese_del_anno'] = df_reset['DATE_TIME'].dt.month
df_elec = df_reset.set_index('DATE_TIME')

In [ ]:
# 3 BOXPLOT
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

df_elec.boxplot(column='Demand', by='giorno_settimana', ax=axes[0])
axes[0].set_xlabel('Giorno della settimana')
axes[0].set_ylabel('Domanda di elettricità')

df_elec.boxplot(column='Demand', by='ora_del_giorno', ax=axes[1])
axes[1].set_xlabel('Ora del giorno')
axes[1].set_ylabel('Domanda di elettricità')

df_elec.boxplot(column='Demand', by='mese_del_anno', ax=axes[2])
axes[2].set_xlabel("Mese dell'anno")
axes[2].set_ylabel('Domanda di elettricità')

plt.tight_layout()
plt.suptitle('')  
plt.show()

#### **Favorita sales**

In [ ]:
train = pd.read_csv("data/favorita_train.csv")
test = pd.read_csv("data/favorita_test.csv")
train.drop(columns=["id"], inplace=True)
test.drop(columns=["id"], inplace=True)
train.rename(columns={"date":"DATE_TIME"}, inplace=True)
train['DATE_TIME'] = pd.to_datetime(train['DATE_TIME'], format='%Y-%m-%d')
test.rename(columns={"date":"DATE_TIME"}, inplace=True)
test['DATE_TIME']= pd.to_datetime(test['DATE_TIME'], format='%Y-%m-%d')
train.set_index("DATE_TIME", inplace=True)
test.set_index("DATE_TIME", inplace=True)
train.head(2)

In [ ]:
#plot con ts_plotly per store 1 e 50
series_dict = {
    'Store 50': train[train["store_nbr"]==50]["sales"],
    'Store 1': train[train["store_nbr"]==1]["sales"]
    
}
ts_plotly(series_dict, title="Sales Store 1 and Store 50")


3 milioni di record. Vendite di alcuni supermercati Favorita in Ecuador. Abbiamo l'identificativo di store, il tipo di prodotto, le vendite totali per un prodotto in uno store in un giorno, e la variabile 'onpromotion'

#### **Rossman sales** (in pausa)

In [ ]:
train = pd.read_csv('./data/ROSSMAN_train.csv', index_col='Date', parse_dates = True)

In [ ]:
series_dict = {
    'sales store 1':train[train["Store"]==1]['Sales'],
    'sales store 2':train[train["Store"]==2]['Sales'],
    'sales store 3':train[train["Store"]==3]['Sales']
}
ts_plotly(series_dict, title="Sales")

Serie storica delle vednite in 1k+ supermercati Rossman. L'idea è di fare analisi panel con questa e Favorita.

In [ ]:
series_dict = {
    'sales store 3':train[train["Store"]==88]['Sales']
}
ts_plotly(series_dict, title="Sales")

In [ ]:
train[train["Store"]==88].shape

Possiamo utilizzare la serie univariata di un singolo magazzino per outlier/NaN